![Coronavirus particle Image](img.png)

### Done By  
* Dr.Nawras Shatnawi
* Osama AbuAl-Zulof
* Hamzah Al-Magarba
* Hussam Al-Saaideh
* Asem Al-Zaghateet

## What is COVID-19?

> Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as Severe Acute Respiratory Syndrome (SARS) and Middle Eastern Respiratory Syndrome (MERS).1
'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named SARS-CoV-2 by the World Health Organization (WHO). The disease it causes has been named “coronavirus disease 2019” (or “COVID-19”).`


In [10]:
# Imports packages

from IPython.core.display import display, HTML 
from IPython.display import YouTubeVideo

import ipywidgets 
from ipywidgets import interact

import folium
from folium import plugins

import pandas as pd
import numpy as np
import json

import plotly as py
import plotly.graph_objects as go
import plotly.express as px

In [3]:
# load Covid_data

cd=pd.read_csv('https://raw.githubusercontent.com/osamazalaf/covid19jordan/main/data/covidmap.csv')
cd1=pd.read_csv('https://raw.githubusercontent.com/osamazalaf/covid19jordan/main/data/covidmap.csv')

cd2=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

cd3=pd.read_csv('https://raw.githubusercontent.com/osamazalaf/covid19jordan/main/data/Covidss.csv')

cd4=pd.read_csv('https://raw.githubusercontent.com/osamazalaf/covid19jordan/main/data/Coviddaily.csv')

cd5=pd.read_csv('https://raw.githubusercontent.com/osamazalaf/covid19jordan/main/data/Monthly.csv')


In [5]:
cd20=cd2[cd2.Country_Region == 'Jordan']

# total number of confirmed, death , recovered and active cases
confirmed_total = int(cd20['Confirmed'])
deaths_total = int(cd20['Deaths'])
recovered_total = int(cd20['Recovered'])
active_total = int(cd20['Active'])

# displaying the total stats

display(HTML("<div style = 'background-color: #111111; padding: 30px '>" +
         "<span style='color: red; font-size:24px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
         "<span style='color: #fff; font-size:24px;margin-left:25px;'> Deaths: " + str(deaths_total) + "</span>"+
         "<span style='color: lightgreen; font-size:24px; margin-left:25px;'> Recovered: " + str(recovered_total) + "</span>"+
         "<span style='color: lightblue; font-size:24px; margin-left:25px;'> Active: " + str(active_total) + "</span>"+
         "</div>"))


In [11]:
cd1["Confirmed"]=np.log(cd1["Confirmed"])

# Create map with location start point, zoom level and distance scale
map = folium.Map(location=[31.5,36.5], zoom_start=6.50,control_scale=True)
                
# load geo_json
country_info = json.load(open('gov.json', encoding='utf-8'))


choropleth = folium.Choropleth(
    geo_data=country_info,
    name='Jordan Confirmed Cases',
    data=cd1,
    columns=['OBJECTID', 'Confirmed'],
    # see folium.Choropleth? for details on key_on
    key_on='feature.properties.OBJECTID',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    line_opacity=0.5,
    legend_name='Confirmed Cases',
    highlight=True
).add_to(map)


tooltip_text = []
for i in range(0,len(cd)):
    tooltip_text.append('<li> City: '+str(cd.iloc[i]['City'])+
                          '<li> Total Confirmed: '+str(cd.iloc[i]['Confirmed'])+
                           '<li> Today Confirmed Cases: '+str(cd.iloc[i]['today'])+
                           '<li> Top Confirmed Cases: '+str(cd.iloc[i]['top Confirmed']))
tooltip_text
           
# Append a tooltip column with customised text
for i in range(0,len(tooltip_text)):
    country_info['features'][i]['properties']['tooltip1'] = tooltip_text[i]
    
    
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['tooltip1'], labels=False))
  

# add tiles to map
folium.raster_layers.TileLayer('Open Street Map').add_to(map)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(map)
  
    
# measure control
measure_control = plugins.MeasureControl(position='topleft', 
                                         active_color='red', 
                                         completed_color='red', 
                                         primary_length_unit='kilometers')

# add measure control to map
map.add_child(measure_control)

# draw tools
draw = plugins.Draw(export=True)

# add draw tools to map
draw.add_to(map)


minimap=plugins.MiniMap(toggle_display=True,auto_toggle_display=True,minimized=False)
map.add_child(minimap)


# add full screen button to map
plugins.Fullscreen(position='bottomright').add_to(map)

folium.LayerControl(position='bottomright').add_to(map)


# put in path.html
#map.save('map.html')

# display map
display(map)


In [21]:
def plot_cases_of_a_country(City):
    labels = ['confirmed']
    colors = ['Red']
    mode_size = [6, 8]
    line_size = [1, 2]
    
    df_list = [cd3]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        
        if City == 'Jordan':
            x_data = np.array(list(df.iloc[:,:].columns))
            y_data = np.sum(np.asarray(df[df['City'] == City].iloc[:,:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, :].columns))
            y_data = np.sum(np.asarray(df[df['City'] == City].iloc[:,:]),axis = 0)
        
        
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True));
    
    fig.update_layout(
        title="Daily Confirmed COVID 19 Cases of " + City,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        height=700)
    
    fig.show();
    
    
s=ipywidgets.Dropdown(options=['Jordan','Mafraq','Irbid','Jerash','Ajloun','Zarqa','Amman','Balqa','Madaba',
                               'Karak','Tafilah','Maan','Aqaba'],
                     value='Jordan',description='City',disabled=False) 

ipywidgets.interact(plot_cases_of_a_country, City=s)

ipywLayout = ipywidgets.Layout(border='solid 2px green')


interactive(children=(Dropdown(description='City', options=('Jordan', 'Mafraq', 'Irbid', 'Jerash', 'Ajloun', '…

In [8]:
idth=Salidapie = ipywidgets.Output()

with Salidapie:
    fig3 = px.pie(cd, values='Confirmed', names='City', color="Confirmed", hole=0.3,height=475)

    fig3.update_traces(textinfo="label+percent", insidetextfont=dict(color="white"))

    #fig3.update_layout(legend={"itemclick":'toggle'})

    fig3.update_layout(title='Confirmed Cases Percentage of Covid-19 in Cities')
    
    fig3 = go.FigureWidget(fig3)
    display(fig3)
    
    
Salidapies = ipywidgets.Output()    
    
with Salidapies:
    v1=int(cd20['Confirmed'])/10285867
    v2=int(cd20['Deaths'])/10285867
    v3=int(cd20['Recovered'])/10285867
    v4=(9579512/10285867)
    v=[v1,v2,v3,v4]

    fig33 = px.pie(values=v, names=['Confirmed','Deaths',"Recovered","Non Confirmed"] ,hole=0.3,height=475)

    fig33.update_traces(textinfo="label+percent", insidetextfont=dict(color="white"),hovertemplate=None,hoverinfo='none')

    #fig3.update_layout(legend={"itemclick":'toggle'})

    fig33.update_layout(title='Cases Percentage of Covid-19 in Jordan')
    
    fig33 = go.FigureWidget(fig33)
    display(fig33)

    
Salidas = ipywidgets.HBox([Salidapie,Salidapies])

display(Salidas)


In [23]:
def pic(m):
    if m=='All':
        fig5 = px.bar(cd5, x='Confirmed', y='City', color='City',height=575,log_x=True)
        fig5.update_layout(title='The Confirmed Cases of Covid-19 in Jordan')
        fig5 = go.FigureWidget(fig5)
        display(fig5)
    
    elif m=='Sep(20)':
        fig5 = px.bar(cd5, x='Sep(20)', y='City', color='City',height=575,log_x=True)
        fig5.update_layout(title='In '+m+' The Confirmed Cases of Covid-19 in Jordan')
        fig5 = go.FigureWidget(fig5)
        display(fig5)
        
    elif m=='Oct(20)':
        fig5 = px.bar(cd5, x='Oct(20)', y='City', color='City',height=575,log_x=True)
        fig5.update_layout(title='In '+m+' The Confirmed Cases of Covid-19 in Jordan')
        fig5 = go.FigureWidget(fig5)
        display(fig5)
        
    elif m=='Nov(20)':
        fig5 = px.bar(cd5, x='Nov(20)', y='City', color='City',height=575,log_x=True)
        fig5.update_layout(title='In '+m+' The Confirmed Cases of Covid-19 in Jordan')
        fig5 = go.FigureWidget(fig5)
        display(fig5)
        
    elif m=='Dec(20)':
        fig5 = px.bar(cd5, x='Dec(20)', y='City', color='City',height=575,log_x=True)
        fig5.update_layout(title='In '+m+' The Confirmed Cases of Covid-19 in Jordan')
        fig5 = go.FigureWidget(fig5)
        display(fig5)
        
    elif m=='Jan(21)':
        fig5 = px.bar(cd5, x='Jan(21)',y='City', color='City',height=575,log_x=True)
        fig5.update_layout(title='In '+m+' The Confirmed Cases of Covid-19 in Jordan')
        fig5 = go.FigureWidget(fig5)
        display(fig5)
        
    elif m=='Feb(21)':
        fig5 = px.bar(cd5, x='Feb(21)', y='City', color='City',height=575,log_x=True)
        fig5.update_layout(title='In'+m+' The Confirmed Cases of Covid-19 in Jordan')
        fig5 = go.FigureWidget(fig5)
        display(fig5)

        
d=ipywidgets.Dropdown(options=['All','Sep(20)','Oct(20)','Nov(20)','Dec(20)','Jan(21)','Feb(21)'],
                     value='All',description='Month',disabled=False)

ipywidgets.interact(pic,m=d)
ipywLayout = ipywidgets.Layout(border='solid 2px green')


interactive(children=(Dropdown(description='Month', options=('All', 'Sep(20)', 'Oct(20)', 'Nov(20)', 'Dec(20)'…

In [24]:
c1=cd4[cd4['Confirmed']>1000]
c2= cd4[(cd4['Confirmed']>250) & (cd4['Confirmed']<500)]
c3= cd4[(cd4['Confirmed']>500) & (cd4['Confirmed']<1000)]
c4= cd4[(cd4['Confirmed']>0) & (cd4['Confirmed']<250)]


def buble(Cases):
    if Cases=='500<Cases<1000' :
        fig2 = px.scatter(c3, y="Date", x="Confirmed",
                size="Confirmed", color="City",
                log_x=True,size_max=40,title='Jordan Covid19 Daily Cases Compare',height=700)
        
        fig2=go.FigureWidget(fig2)
        display(fig2)
        
    elif Cases=='Cases>1000':
        fig2 = px.scatter(c1, y="Date", x="Confirmed",
                size="Confirmed", color="City",
                log_x=True,size_max=45,title='Jordan Covid19 Daily Cases Compare',height=700)
       
        fig2=go.FigureWidget(fig2)
        display(fig2)
        
    elif Cases=='All Cases':
        fig2 = px.scatter(cd4, y="Date", x="Confirmed",
                size="Confirmed", color="City",
                log_x=True,size_max=32.5,title='Jordan Covid19 Daily Cases Compare',height=725)
       
        fig2=go.FigureWidget(fig2)
        display(fig2)
        
    elif Cases=='250<Cases<500':
        fig2 = px.scatter(c2, y="Date", x="Confirmed",
                size="Confirmed", color="City",
                log_x=True,size_max=35,title='Jordan Covid19 Daily Cases Compare',height=700)
       
        fig2=go.FigureWidget(fig2)
        display(fig2)
        
    elif Cases=='0<Cases<250':
        fig2 = px.scatter(c4, y="Date", x="Confirmed",
                size="Confirmed", color="City",
                log_x=False,size_max=22.5,title='Jordan Covid19 Daily Cases Compare',height=725)
       
        fig2=go.FigureWidget(fig2)
        display(fig2)
        
        
f=ipywidgets.Dropdown(options=['All Cases','0<Cases<250','250<Cases<500','500<Cases<1000','Cases>1000'],
                     value='All Cases',description='Range',disabled=False)

ipywidgets.interact(buble,Cases=f)
       
ipywLayout = ipywidgets.Layout(border='solid 2px green')


interactive(children=(Dropdown(description='Range', options=('All Cases', '0<Cases<250', '250<Cases<500', '500…

### To See World Active Cases Put (N < Zero)

In [25]:
def pics(N):
    if N>0:
        cd30=cd2[cd2['Active']<N]
        fig4 = go.Figure(data=go.Choropleth(
            locations=cd30['Country_Region'], # Spatial coordinates
            z = cd30['Active'].astype(float), # Data to be color-coded
            locationmode = 'country names', # set of locations match entries in `locations`
            colorscale='YLOrRd',zmin=0,zmax=N));
        
        fig4.update_layout(
        title_text = 'Active Covid-19 Cases Last Day In Asia',
        geo_scope='asia',height=500);
        
        fig4.update_geos(fitbounds='locations',visible=False);
        fig4=go.FigureWidget(fig4);
        fig4.show();
        
    elif N<0:
        cd30=cd2[cd2['Active']<-N]
        fig4 = go.Figure(data=go.Choropleth(
            locations=cd30['Country_Region'], # Spatial coordinates
            z = cd30['Active'].astype(float), # Data to be color-coded
            locationmode = 'country names', # set of locations match entries in `locations`
            colorscale='YLOrRd',zmin=0,zmax=-N));
        
        fig4.update_layout(
        title_text = 'Active Covid-19 Cases Last Day In World',
        geo_scope='world');
        
        fig4.update_geos(fitbounds='locations',visible=False);
        fig4=go.FigureWidget(fig4);
        fig4.show();
        

        
ipywidgets.interact(pics,N=(-5500000,5750000))

ipywLayout = ipywidgets.Layout(border='solid 2px green')


interactive(children=(IntSlider(value=125000, description='N', max=5750000, min=-5500000), Output()), _dom_cla…

In [29]:
SalidaVideo= ipywidgets.Output()

with SalidaVideo:
    display(YouTubeVideo(id='7e3oAU419k8',width=450,height=450))
    
SalidaVideos = ipywidgets.Output()    
    
with SalidaVideos:
    display(YouTubeVideo(id='M0BRsaIsr30',width=450,height=450))

Salidas = ipywidgets.HBox([SalidaVideo,SalidaVideos])

display(Salidas)


# Notebook covers

1. What Is COVID-19? [Covid wikipedia](https://en.wikipedia.org/wiki/Coronavirus_disease_2019)
2. Data Loading From [Ministry of Health The Hashemite kingdom of Jordan](https://corona.moh.gov.jo/ar)


## Symptoms
People may be sick with the virus for 1 to 14 days before developing symptoms. The most common symptoms of coronavirus disease (COVID-19) are fever, tiredness, and dry cough. Most people (about 80%) recover from the disease without needing special treatment.
* cough
* fever
* tiredness
* difficulty in breathing(severe cases)



## More Info on COVID-19 for Other Countries:
* [www.worldometers.info/coronavirus](https://www.worldometers.info/coronavirus/)


## Our World Covid Tracker App 
* [Download World Covid Tracker(in Pc)](https://drive.google.com/u/1/uc?id=169NAaty25FRcmwHABiLtjjoDusOmwxFa&export=download)



## If You Want The Site Send You Lastest Info about Covid-19 in Jordan Via Email
* [Please Enter Your Email Here](https://docs.google.com/forms/d/1j5K6ow5ru9q-Lfd3Qd5LGbcUaHOv5r-8YfOfJmQpWQk/edit)


## Contact Us By The Following Email
   * Coviid19jo@gmail.com

### Done By
* Osama AbuAl-Zulof
* Hamzah Al-Magarba
* Hussam Al-Saaideh
* Asem Al-Zaghateet

### Supervision By 
* Dr.Nawras Shatnawi
